# federated-ml-health 
**Notatnik przystosowany do zajęć z SIwIB**.
* aktualizacja do nowszej wersji biblioteki TF Federated
* uproszczenie kodu (usunięcie części związanej z *Differential Privacy*) i przygotowanie fragmentów kodu na potrzeby zajęć

---

Oryginalna wersja: https://github.com/google/federated-ml-health

Copyright 2020 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

In [ ]:
# Instalacja TF Federated
%pip install --quiet --upgrade tensorflow-federated




# Przygotowanie danych

Na początku wykorzystamy zbiór `pima`. W dalszej kolejności będziemy pracować na odpowiednio przygotowanej wersji zbioru MIMIC-III (dostępny na eKursach).

In [ ]:
import collections
import matplotlib.pyplot as plt
import nest_asyncio
import numpy as np
import pandas as pd
import sklearn
import tensorflow as tf
import tensorflow_federated as tff
from collections import defaultdict 
from matplotlib.pyplot import figure
from numpy import loadtxt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [ ]:
# Ukrycie części niestotnych logów
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [ ]:
# Podmontowanie dysku Google
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)


In [ ]:
# Ścieżki do plików przy założeniu, że pliki z danymi znajdują się w głównym katalogu na GDrive
csv_fn = '/content/drive/MyDrive/pima.csv' 
# W dalszej kolejności będziemy używać '/content/drive/MyDrive/mimic3.csv'
raw_ds = pd.read_csv(csv_fn)
num_col = raw_ds.shape[1]
# Zakładamy, że atrybut decyzyjny jest zawsze w ostatniej kolumnie
X = raw_ds.iloc[:, 0:num_col-1].values
y = raw_ds.iloc[:, num_col-1].values

## Podział na część uczącą i testującą

Tym razem porządniej, niż w oryginalnym notatniku :) -- `scaler` oraz `imputer` są uczone na danych uczących i stosowane do danych testowych.

In [ ]:
TRAIN_PROPORTION = 0.8
NUM_FEATURES = X.shape[1]
NUM_ROUNDS = 12

n_train = round(TRAIN_PROPORTION * X.shape[0])
n_test = X.shape[0] - n_train

X_train = X[:n_train]
y_train =  y[:n_train]
X_test = X[n_train:]
y_test =  y[n_train:]

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline 

preprocessor = make_pipeline(SimpleImputer(), StandardScaler())

X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)


# Podejście scentralizowane

## Regresja - scikit-learn


Testujemy kilka wariantów regresji logistycznej, aby uzyskać *baseline*.

In [ ]:
sk_model = LogisticRegression(random_state=42).fit(X_train, y_train)
proba_test = sk_model.predict_proba(X_test)[:,1]
fpr_sk, tpr_sk, threshold_sk = sklearn.metrics.roc_curve(y_test, proba_test)
auc_sk = sklearn.metrics.auc(fpr_sk, tpr_sk)
print(f'AUC-LIN = {roc_auc_sk:.4}')

## Regresja - TF

Implementujemy regresję logistyczną w TF. Stworzony model (`tf_model`) wykorzystuje `Adam`-a. Wyjaśnienie autorów notatnika: *Adam optimization method is used to mimic the sklearn solver as close as possible (leveraging second derivatives of gradient).*

In [ ]:
# dataset_train = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(n_train)
# dataset_test = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(n_train)

In [ ]:
# Prosty model - LR
def create_keras_model():
  return tf.keras.models.Sequential([
      tf.keras.layers.Dense(
          1,
          activation='sigmoid',
          input_shape=(NUM_FEATURES,),
          kernel_regularizer=tf.keras.regularizers.l2(0.01),
      )
  ])

# Bardziej złożony model - MLP
def create_keras_model_deeper():
  initializer = tf.keras.initializers.GlorotNormal(seed=10)
  m = tf.keras.models.Sequential()
  m.add(tf.keras.Input(shape=(NUM_FEATURES,)))
  m.add(tf.keras.layers.Dense(6, activation='sigmoid', kernel_initializer=initializer))
  m.add(tf.keras.layers.Dense(3, activation='sigmoid', kernel_initializer=initializer))
  m.add(tf.keras.layers.Dense(1, activation='sigmoid', kernel_initializer=initializer, kernel_regularizer=tf.keras.regularizers.L1L2(l1=0.0001, l2=0.01)))
  return m
  

In [ ]:
tf_model = create_keras_model()
tf_model.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=0.5),   
  loss=tf.keras.losses.BinaryCrossentropy(),
  metrics=[
    tf.keras.metrics.BinaryAccuracy(name='accuracy'),
    tf.keras.metrics.AUC(name='auc'),
  ]
)

batch_size = round(n_train/3)


In [ ]:
# tf_model.fit(dataset_train, validation_data=dataset_test, epochs=NUM_ROUNDS, batch_size=batch_size, verbose=1, use_multiprocessing=True)

tf_model.fit(X_train, y_train, epochs=NUM_ROUNDS, batch_size=batch_size, verbose=1, use_multiprocessing=True)


In [ ]:
proba_test = tf_model.predict(X_test)
fpr_tf, tpr_tf, threshold = sklearn.metrics.roc_curve(y_test, proba_test)
auc_tf = sklearn.metrics.auc(fpr_tf, tpr_tf)
print(f'AUC-TF = {auc_tf:0.4}')

# Regresja - TF Federated

Utworzenie zbioru z danymi uczącymi, aby ułatwić przydział danych do poszczególnych klientów

In [ ]:
df_X_train = pd.DataFrame(data=X_train, columns=raw_ds.columns[:-1])
df_y_train = pd.DataFrame(data=y_train, columns=raw_ds.columns[-1:])

Przypisanie identyfikatorów (indeksów) przykładów uczących do poszczególnych klientów. Obecnie wszyscy klienci otrzymują taką samą liczbę przykładów, przy czym rozkład klas nie jest zachowywany. Ta funkcja powinna zostać zmodyfikowna w ramach projektu.

In [ ]:
def assign_samples_to_clients(data, n_clients):
    from sklearn.model_selection import KFold
    client_sample_ids = []
    kf = KFold(n_splits=n_clients, shuffle=True, random_state=42)
    for _, test_ids in kf.split(data):
        client_sample_ids.append(test_ids)
    return client_sample_ids

In [ ]:
NUM_CLIENTS = 20
# NUM_PARTICIPATING_PER_ROUND = round(NUM_CLIENTS/3)

In [ ]:
client_ids = list(range(NUM_CLIENTS))
client_sample_ids = assign_samples_to_clients(X_train, NUM_CLIENTS)

In [ ]:
def create_client_dataset(data, labels, client_ids, client_sample_ids):
  def create_dataset_fn(client_id):
    sample_ids = client_sample_ids[client_id]
    return tf.data.Dataset.from_tensor_slices((data[sample_ids, :], labels[sample_ids]))

  return tff.simulation.datasets.ClientData.from_clients_and_tf_fn(
      client_ids=client_ids,
      serializable_dataset_fn=create_dataset_fn)
  
def preprocess(dataset):
    card = dataset.cardinality()
    batch_size = 1 if card == tf.data.INFINITE_CARDINALITY or tf.data.UNKNOWN_CARDINALITY else round(card.numpy()/3)
    return dataset.batch(batch_size)

def make_federated_data(client_data, client_ids):
  return [
      preprocess(client_data.create_tf_dataset_for_client(id))
      for id in client_ids
  ]

In [ ]:
client_dataset_train = create_client_dataset(X_train, y_train, client_ids, client_sample_ids)

In [ ]:
spec_dataset = preprocess(client_dataset_train.create_tf_dataset_for_client(client_ids[0]))

def model_fn():
  keras_model = create_keras_model()
  return tff.learning.models.from_keras_model(
    keras_model,
    input_spec=spec_dataset.element_spec,
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=[
        tf.keras.metrics.BinaryAccuracy(name='accuracy'),
        tf.keras.metrics.AUC(name='auc')
    ]
  )
  
# Tworzymy iteracyjny proces uczący z wykorzystaniem bazowego algorytmu FedAvg
trainer = tff.learning.algorithms.build_weighted_fed_avg(
    model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.Adam(learning_rate=0.5), 
    server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=1.0),
    use_experimental_simulation_loop = True
)

In [ ]:
tff_auc = defaultdict(lambda:0)

In [ ]:
# Zewnętrzna pętla pozwala na sprawdzanie różnej liczby klientów biorących udział w każdej rundzie obliczeń.
# Na potrzeby projektu należy założyć, że wszyscy klienci uczestniczą w obliczeniach

possible_num_clients_per_round = list(range(2, NUM_CLIENTS, 4))

if not NUM_CLIENTS in possible_num_clients_per_round:
  possible_num_clients_per_round.append(NUM_CLIENTS)

for num_clients_per_round in possible_num_clients_per_round:
  print(f"# participating clients = {num_clients_per_round}")
  
  state = trainer.initialize()
  tff_model = create_keras_model()

  for r in range(NUM_ROUNDS):
    participating_client_ids = np.random.choice(range(NUM_CLIENTS), size=num_clients_per_round, replace=False)
    print(f"round {r + 1}/{NUM_ROUNDS} | participants = {participating_client_ids}")
    federated_train_data = make_federated_data(client_dataset_train, participating_client_ids)
    state, metrics = trainer.next(state, federated_train_data)
    # print(n_clients, i_round, str(metrics))

  weights = trainer.get_model_weights(state)
  weights.assign_weights_to(tff_model)

  proba_test = tff_model.predict(X_test)
  fpr_test, tpr_test, _ = sklearn.metrics.roc_curve(y_test, proba_test)
  auc_test = sklearn.metrics.auc(fpr_test, tpr_test)
  loss_test = tf.keras.losses.binary_crossentropy(y_test, np.reshape(proba_test, [-1]))
  print(f'AUC = {auc_test:0.4}, Loss={loss_test:0.4}')

  tff_auc[num_clients_per_round] = (auc_test, fpr_test, tpr_test)


### Porównanie stworzonych modeli

In [ ]:
figure(num=None, figsize=(8, 6), dpi=150, facecolor='w', edgecolor='k')
plt.title('ROC')
plt.plot(fpr_sk, tpr_sk, label = f'sk-LR AUC = {auc_sk:0.3f}')
plt.plot(fpr_tf, tpr_tf, label = f'tf-LR (centralized) AUC = {auc_tf:0.3f}')
# Wyniki dla podejścia sfederowanego
for num_participants, (auc_tff, fpr_tff, tpr_tff) in tff_auc.items():
  plt.plot(fpr_tff, tpr_tff, label = f'tff-LR (federated, p = {num_participants}) AUC = {auc_tff:0.3f}')
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('TPR')
plt.xlabel('FPR')
plt.show()